In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import torch
import torchvision # image processing에 특화된 라이브러리
import torch.nn as nn
from torchvision import transforms # Data Augmentation에 특화된 라이브러리
import numpy as np

In [3]:
X = torch.Tensor(2, 3) # 2행 3열의 텐서를 하나 생성
X.shape # == X.size()
X

torch.Size([2, 3])

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [4]:
X = torch.Tensor([[1, 2, 3], [4, 5, 6]])
X

tensor([[1., 2., 3.],
        [4., 5., 6.]])

### tensor()인자값으로  
    data : 값 지정, 초기화  
    dtype  
    requires_grad : tensor에 대해서 미분 적용할지 여부  

In [5]:
x = torch.tensor(data=[2.0, 3.0], requires_grad=True)
x

y = x**2
y

pred = 2*y + 3
pred

target = torch.tensor([3.0, 4.0])
target

tensor([2., 3.], requires_grad=True)

tensor([4., 9.], grad_fn=<PowBackward0>)

tensor([11., 21.], grad_fn=<AddBackward0>)

tensor([3., 4.])

### 위에서 나온 결과를 바탕으로 경사하강법을 이용해서 Loss를 줄여나가 보겠다.

In [6]:
loss = torch.sum(torch.abs(pred - target))
loss

tensor(25., grad_fn=<SumBackward0>)

In [7]:
loss.backward()

In [8]:
x.grad
pred.grad

tensor([ 8., 12.])

<ipython-input-8-7755ea398175>:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  pred.grad


### Create Tensor...Using NeuralNet

In [9]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

x
y

tensor([[-0.0965,  0.4992, -0.8314],
        [ 0.3474, -0.7540,  0.0501],
        [ 0.6691, -0.1490,  1.9303],
        [ 0.8932, -1.4286, -0.7356],
        [-1.5805, -0.1793, -0.2737],
        [-0.2764, -1.3771, -1.0279],
        [-1.7323,  0.4801, -0.1461],
        [ 0.5173, -0.1094, -0.9523],
        [-0.8775, -0.9870,  0.3024],
        [ 0.8129,  0.4990, -1.3129]])

tensor([[ 0.4101,  0.0373],
        [-0.7210,  0.8413],
        [ 0.3187, -0.3910],
        [-0.3194, -0.5916],
        [ 0.5205,  0.4137],
        [-0.7573, -0.4400],
        [-1.8787, -0.5302],
        [ 0.8019, -0.1632],
        [ 0.6345, -1.4639],
        [-0.3875,  0.2452]])

In [10]:
linear = nn.Linear(3, 2)
linear.weight # == w
linear.bias # == b

Parameter containing:
tensor([[-0.2811, -0.0506, -0.2927],
        [ 0.4379,  0.1187,  0.3818]], requires_grad=True)

Parameter containing:
tensor([0.4287, 0.1205], requires_grad=True)

In [11]:
# 객체 출력, parameters()함수는 모델 안의 학습 주체인 w, b를 내포하고 있는 객체
# parameters()함수는 w, b를 해킹한 함수이다
linear.parameters()

<generator object Module.parameters at 0x000002634B374200>

In [12]:
# loss function을 미리 정의
loss_function = nn.MSELoss()

# optimizer(하산하는 방법)를 미리 정의
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# 위에서 만든 모델에 값을 입력...결과로 예측 값이 나온다
pred = linear(x)

# 정답과 예측값을 이용해 위에서 미리 정의한 Loss를 계산...L(w)
loss = loss_function(pred, y)
loss.item()

0.9968193769454956

loss값이 나왔다는 것은  
모델의 예측 값에 대한 잘잘못을 정량화했다는 것이다.  
  
이 값을 바탕으로 BackPropagation을 하면 w, b에 대한 미분값  
즉 얼마만큼 보정해야 하는지가 나온다.  
그 값을 이용해 다시 하강(기울기 수정)하기 때문에  
2번째에는 Loss가 줄어들어야 한다.

In [13]:
loss.backward() # loss값에 대한 w의 책임을 묻는다...미분을 적용, grad
print("dL/dw", linear.weight.grad)
print("dL/db", linear.bias.grad)

dL/dw tensor([[-0.4752, -0.2189, -0.5201],
        [ 0.2220, -0.1781,  0.3561]])
dL/db tensor([0.7092, 0.1107])


In [14]:
optimizer.step() # 위에서 수정된 값으로 하강을 진행...학습

# 반복효과
pred = linear(x)
loss = loss_function(pred, y)

print("loss afer BackPropagation :", loss.item())

loss afer BackPropagation : 0.9842244982719421
